# Classification Analysis of Patient Parameters on Stroke Predictions

In [ ]:
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)

install.packages("themis")
library(recipes)
library(modeldata)
library(themis)

options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”


## Introduction

## Methods and Results

In [3]:
set.seed(1768)
raw_data_url <- "https://raw.githubusercontent.com/jordanjzhao/dsci-project-proposal/main/data/healthcare-dataset-stroke-data.csv"
stroke_data <- read.csv(raw_data_url, na.strings = c("N/A", na)) %>%
    mutate(stroke = as_factor(stroke))

stroke_data_clean <- na.omit(stroke_data)
head(stroke_data_clean)

# Select data
stroke_data_select <- stroke_data_clean %>%
    select(stroke, bmi, avg_glucose_level)
stroke_data_select

# Create training and testing set
stroke_split <- initial_split(stroke_data_select, prop = 0.75, strata = stroke)
stroke_train <- training(stroke_split)
stroke_test <- testing(stroke_split)

ERROR: Error in read.csv(raw_data_url, na.strings = c("N/A", na)) %>% mutate(stroke = as_factor(stroke)): could not find function "%>%"


In [2]:
# scale and standardize predictors
# Due to imbalance in count, more non stroke than stroke cases, must account for this
#stroke_recipe <- recipe(stroke ~ bmi + avg_glucose_level, data = stroke_train) %>%
#    step_scale(all_predictors()) %>%
#    step_center(all_predictors()) %>%
#    step_upsample(stroke, over_ratio = 1, skip = FALSE) %>%
#    prep()

stroke_recipe <- recipe(stroke ~ bmi + avg_glucose_level, data = stroke_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors()) %>%
    step_upsample(stroke, over_ratio = 1, skip = FALSE) %>%
    prep()
stroke_recipe

upsampled_stroke <- bake(stroke_recipe, stroke_train)

upsampled_stroke %>%
    group_by(stroke) %>%
    summarize(n = n())


# 5-fold cross-valiation on training set
stroke_vfold <- vfold_cv(stroke_train, v = 5, strata = stroke)


# KNN classifier
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
    set_engine("kknn") %>%
    set_mode("classification")

# kvals
k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

# Create workflow analysis with recipe and model specs 
stroke_knn_results <- workflow() %>%
    add_recipe(stroke_recipe) %>%
    add_model(knn_tune) %>%
    tune_grid(resamples = stroke_vfold, grid = k_vals) %>%
    collect_metrics()
stroke_knn_results

# plot k vs accuracy 
stroke_knn_accuracies <- stroke_knn_results %>%
    filter(.metric == "accuracy")
stroke_knn_accuracies

# plot
cross_val_plot <- stroke_knn_accuracies %>%
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    theme(text = element_text(size = 12))
cross_val_plot


ERROR: Error in recipe(stroke ~ bmi + avg_glucose_level, data = stroke_train) %>% : could not find function "%>%"


## Discussion

## Works Cited

Boehme, Amelia K., et al. “Stroke Risk Factors, Genetics, and Prevention.” Circulation Research, 3 Feb. 2017, https://www.ahajournals.org/doi/full/10.1161/CIRCRESAHA.116.308398. 

Hägg-Holmberg, Stefanie, et al. “The Role of Blood Pressure in Risk of Ischemic and Hemorrhagic Stroke in Type 1 Diabetes - Cardiovascular Diabetology.” BioMed Central, BioMed Central, 9 July 2019, https://cardiab.biomedcentral.com/articles/10.1186/s12933-019-0891-4. 

Dataset source: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset
Author credentials: fedesoriano